In [368]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [369]:
product_type = "frozen"
num_time_periods = 7

In [370]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
# article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]

In [371]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

In [372]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [373]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [374]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [375]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [376]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start='2022-06-13', end='2022-06-18', freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

#### Heuristic

In [377]:
# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

#### Method 2

In [378]:
demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [379]:
def silverMeal(item):
    
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    life = shelf_life[item]
    # item_demand = [2,3,30,5,3,10]
    # fixed_cost = 10
    # variable_cost = 1
    # cu_per_tu_item = 2
    # life = 2
    # disposal_cost = 0
    orders = {}
    # Start of the date on which we want to order 
    order_date = 0 
    run_complete = False
    obj_val = 0
    while(not run_complete):
        min_cost = float('inf')
        cum_demand = 0
        for t in range(order_date, len(item_demand)):
            cum_demand += item_demand[t]
            if(t==order_date+life):
                orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)
                order_date = t + 1
                break
            # Calculate the average cost which we want to minimise
            cost = fixed_cost + variable_cost * math.ceil(cum_demand/cu_per_tu_item)             
            avg_cost = cost / (t-order_date+1)
            if(avg_cost <= min_cost):
                min_cost = avg_cost
                # If we reach the last day of the period and we did not find a LM before that, we order all the items on that respective date
                if(t==len(item_demand)-1):
                    orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)
                    if (orders[order_date]*cu_per_tu_item) - cum_demand > 0:
                        cost += ((orders[order_date]*cu_per_tu_item) - cum_demand) * disposal_cost
                    run_complete = True
                    break
            # The first time avg_cost increases we jump to the next period from which we need to order 
            else:
                orders[order_date] = math.ceil((cum_demand- item_demand[t])/cu_per_tu_item)
                if (orders[order_date]*cu_per_tu_item) - cum_demand > 0:
                    cost += ((orders[order_date]*cu_per_tu_item) - cum_demand) * disposal_cost
                order_date = t
                break
        obj_val += cost
        # Iterate over all days of the time period
        if(order_date > len(item_demand)-1):
            break
    
    return (orders, obj_val)
obj_val = 0 # keep track of the objective value
vol = 0

schedule = {}
for item in demand.keys():
    orders, val = silverMeal(item) 
    obj_val += val
    for t in orders.keys():
        if t not in schedule.keys():
            schedule[t] = {}
        if item not in schedule[t].keys():
            schedule[t][item] = 0
        schedule[t][item] += orders[t]
    
    
#print(silverMeal('00ee8964'))
print(obj_val)


13792.399999999996


In [326]:
time_indexes

[0, 1, 2, 3, 4, 5]

In [380]:
buf_cost = 0
vol = 0
for time in range(len(time_indexes)):
    if time in schedule.keys():
        for item in schedule[time].keys():
            vol += schedule[time][item] * cu_per_tu[item] * volume_per_cu[item]
    
    if time > 0:
        for item in demand.keys():
            if time in demand[item].keys():
                vol -= demand[item][time] * cu_per_tu[item] * volume_per_cu[item]

    if vol > warehouse_volume:
        buf_cost += (vol - warehouse_volume) * buffer_cost

buf_cost

181.07494764999998

In [383]:
obj_val + buf_cost

13973.474947649996